In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random as rnd

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

import warnings
warnings.filterwarnings("ignore")

# Any results you write to the current directory are saved as output.

In [1]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
combine = [train,test]

In [1]:
print(train.columns.values)

In [1]:
train.head()

In [1]:
train.info()
print('*'*50)
test.info()

分析数据分布

In [1]:
train.describe()

In [1]:
train.describe(include=[np.object])
#train.describe(include=['O'])

## 相关性分析

简单的从数据中看出相关性（无图）

In [1]:
train[['Pclass','Survived']].groupby(['Pclass'],as_index=False)['Survived'].mean()

In [1]:
train[['Sex','Survived']].groupby(['Sex'],as_index=False)['Survived'].mean()

In [1]:
train[['SibSp','Survived']].groupby(['SibSp'],as_index=False)['Survived'].mean().sort_values(
    by='Survived',ascending=False)

In [1]:
train[["Parch", "Survived"]].groupby(['Parch'], as_index=False)['Survived'].mean().sort_values(
    by='Survived', ascending=False)

## 通过图像验证相关性

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [1]:
g = sns.FacetGrid(train,col='Survived')
g.map(plt.hist,'Age',bins=20)

In [1]:
grid = sns.FacetGrid(train,col='Survived',row='Pclass',size=3,aspect=1.5)
grid.map(plt.hist,'Age',alpha=.5,bins=20)
grid.add_legend()

In [1]:
grid = sns.FacetGrid(train, row='Embarked', size=2.2, aspect=1.6)
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex')
grid.add_legend()

Embarked=C 中 男性存活率更高 这表明Pclass和Embarked存在相关性

embarked不一定与存活率有直接关系 可能是 embarked->pclass->survived


In [1]:
grid = sns.FacetGrid(train,row='Embarked',col='Survived')
grid.map(sns.barplot, 'Sex', 'Fare', alpha=.3, ci=None)
grid.add_legend()

In [1]:
train['Cabin'].fillna('U0',inplace = True)
test['Cabin'].fillna('U0',inplace = True)
#train[train['Cabin'] == 'U0']

In [1]:
grid = sns.FacetGrid(train[train['Cabin'] == 'U0'])
grid.map(plt.hist,'Survived',color='r')
grid = sns.FacetGrid(train[train['Cabin'] != 'U0'])
grid.map(plt.hist,'Survived',color='b')

In [1]:
train = train.drop(['Ticket','PassengerId'],axis=1)
test = test.drop(['Ticket','PassengerId'],axis=1)
combine = [train,test]


添加新的特征 改名字

In [1]:
for dataset in combine:
    dataset['Title'] = dataset['Name'].str.extract('([A-Za-z]+)\.',expand=False)
pd.crosstab(train['Title'],train['Sex'])

In [1]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
                                                 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
train[['Title','Survived']].groupby(['Title'],as_index=False).mean().sort_values(by='Survived')

In [1]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)


In [1]:
train = train.drop(['Name'],axis=1)
test = test.drop(['Name'],axis=1)
combine = [train,test]

In [1]:
train.head()

In [1]:
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map({'female':1, 'male':0}).astype(int)
train.head()

完善Age的缺失值

In [1]:
grid = sns.FacetGrid(train,row='Pclass',col='Sex',size=3,aspect=1.6)
grid.map(plt.hist,'Age',bins=25)
grid.add_legend()

利用性别 Pclass 预测年龄

In [1]:
guess_age = np.zeros((2,3))
guess_age

In [1]:
for dataset in combine:
    for i in range(0,2):
        for j in range(0,3):
            guess = dataset[(dataset['Sex']==i) & (dataset['Pclass']==j+1)]['Age'].dropna()
            age_guess = guess.median()
            guess_age[i,j] = int(age_guess/0.5 + 0.5)*0.5
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),'Age'] = guess_age[i,j]

    dataset['Age'] = dataset['Age'].astype(int)
train.head()

In [1]:
train['AgeBand'] = pd.cut(train['Age'],5)
train[['AgeBand','Survived']].groupby(['AgeBand'],as_index=False).mean().sort_values(by='AgeBand', ascending=True)

In [1]:
for dataset in combine:
    dataset.loc[dataset['Age']<=16,'Age'] = 0
    dataset.loc[(dataset['Age']<=32) & (dataset['Age']>16),'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']


In [1]:
train = train.drop(['AgeBand'],axis=1)
combine = [train,test]

In [1]:
for dataset in combine:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass

Parch and SibSp

In [1]:
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
train[['FamilySize','Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [1]:
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

train[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

In [1]:
for dataset in combine:
    dataset['HasCabin'] = 1
    dataset.loc[dataset['Cabin'] == 'U0', 'HasCabin'] = 0

train[['HasCabin', 'Survived']].groupby(['HasCabin'], as_index=False).mean()

In [1]:
train = train.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test = test.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [train,test]
train

处理票价

In [1]:
train['FareBand'] = pd.qcut(train['Fare'],4)
train[['FareBand','Survived']].groupby(['FareBand'],as_index=False).mean().sort_values(by='FareBand', ascending=True)

In [1]:
test['Fare'].fillna(test['Fare'].dropna().median(),inplace=True)

In [1]:
for dataset in combine:
    dataset.loc[dataset['Fare']<=7.91,'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train = train.drop(['FareBand'], axis=1)
combine = [train,test]

In [1]:
freq_port = train.Embarked.mode()[0]
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)

In [1]:
train = train.drop(['Cabin'], axis=1)
test = test.drop(['Cabin'], axis=1)
train = pd.get_dummies(train)
test = pd.get_dummies(test)
combine = [train,test]

In [1]:
train

In [1]:
test

In [1]:
train_x = train.drop('Survived', axis=1)
train_y = train['Survived']
test_x = test
train_x.shape,train_y.shape,test_x.shape

# 使用RF算法

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
random_forest = RandomForestClassifier(n_estimators=100)

results = cross_val_score(random_forest,train_x,train_y,scoring='accuracy',cv=10)
print(np.mean(results))

In [1]:
random_forest.fit(train_x,train_y)
predict = random_forest.predict(test_x)
col = pd.read_csv('../input/test.csv')
output = pd.DataFrame({'PassengerId':col['PassengerId'],'Survived':predict})
output.to_csv('submission.csv',index=False)